In [1]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator
from preprocess import preProcessData

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
point_cloud_location = "/home/chris/Code/PointClouds/data/FLIPscans/Bendy/Bendy_1/scan1_Part1.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

mesh = o3d.io.read_triangle_mesh("/home/chris/Code/PointClouds/data/FLIPscans/Bendy/BendyCAD.STL")
mesh.compute_vertex_normals()

TriangleMesh with 4156 points and 2104 triangles.

In [3]:
# Preprocess the point cloud
pcd, average_density, cad_pcd = preProcessData(pcd, mesh, x_rotation=110, z_rotation=90)
pcd_points = np.asarray(pcd.points)

# Detect planes, intersections, and anchor points
segment_models, segments, segment_indices, main_surface_idx = util.multiOrderRansacAdvanced(pcd, pt_to_plane_dist=0.4, verbose=False, visualize=False)
angles_rad = util.findAnglesBetweenPlanes(segment_models, main_surface_idx)
intersection_lines = util.findIntersectionLinesLeastSquares(segment_models, main_surface_idx)
anchor_points = util.findAnchorPoints(segment_models, segments, intersection_lines, main_surface_idx)

In [4]:
# Callback test

pcd_tree = o3d.geometry.KDTreeFlann(pcd)
eig_val_dict = {}
sample_dist = 0.2
number_of_neighbors = 5
sampled_neighborhoods = {}
initial_eigen_agg = 0
forward = True
#eigen_threshold = 100000
#angle_threshold = 0.11
eigen_threshold = 0.07
angle_threshold = 0.12
radius = 1.5

class CallbackState:
    def __init__(self, anchor_points, pcd_tree, 
                 eigen_threshold,number_of_neighbors, 
                 sample_dist, radius, intersection_lines, pcd, angle_threshold):

        self.pcd_tree=pcd_tree
        self.pcd_color = (0.6,0.6,0.6)
        self.counter = 0  # This parameter changes each time the function is called
        self.forward=True
        self.anchor_points=anchor_points
        self.eigen_threshold=eigen_threshold
        self.angle_threshold=angle_threshold
        self.number_of_neighbors=number_of_neighbors
        self.sample_dist=sample_dist
        self.radius=radius
        self.intersection_lines=intersection_lines
        self.segment_models = segment_models
        self.pcd = pcd
        self.pcd.paint_uniform_color(self.pcd_color)
        self.sampled_neighborhoods={}
        self.coloring_sampled_neighborhoods={}
        self.initial_eigen_agg=0
        self.initial_angle_agg=0
        self.eig_val_dict={}
        self.plane_dir_dict={}
        self.it=0
        self.seg_id = 1
        self.pcd_center = pcd.get_center()
        self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Eigen agg =', depth=4).to_legacy()
        self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text('Angle agg =', depth=4).to_legacy()
        self.line_geometry = []
        self.ellipsoid_geometry = None

    def toggle_ellipsoid(self, vis):
        if self.ellipsoid_geometry:
            vis.remove_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
            self.ellipsoid_geometry = None
        else:
            if self.it-1 in self.sampled_neighborhoods:
                # if self.forward:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] + ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                # else:
                #     sampled_anchor_point = self.anchor_points[self.seg_id] - ((self.it-1)*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
                idx = self.sampled_neighborhoods[self.it-1]
                neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

                mu = np.mean(neighbor_coordinates, axis=0)
                norm = neighbor_coordinates - mu
                cov = np.cov(norm.T)
                eig_val, eig_vec = np.linalg.eig(cov)
                sorted_idx = np.argsort(eig_val)[::-1]
                eig_val = eig_val[sorted_idx]
                eig_vec = eig_vec[:, sorted_idx]

                # Create a sphere mesh
                sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)

                # Scale the sphere to form an ellipsoid based on the eigenvalues
                scale_matrix = 2*np.diag(np.sqrt(eig_val))

                # Construct the 4x4 transformation matrix
                transformation = np.eye(4)
                transformation[:3, :3] = eig_vec @ scale_matrix
                transformation[:3, 3] = mu

                # Apply the transformation to the sphere
                ellipsoid = sphere.transform(transformation)

                # Set the color of the ellipsoid
                ellipsoid.paint_uniform_color([0.2, 0.7, 0.3])  

                self.ellipsoid_geometry = ellipsoid
                vis.add_geometry(self.ellipsoid_geometry, reset_bounding_box=False)
        vis.update_renderer()
    
    def create_cylinder_between_points(self, point1, point2, radius, color):
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=np.linalg.norm(point2 - point1))
        cylinder.paint_uniform_color(color)
        
        # Compute the direction vector and normalize it
        direction = (point2 - point1) / np.linalg.norm(point2 - point1)
        
        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))
        
        # Apply the rotation
        cylinder.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))
        
        # Translate to the start point
        center = (point2+point1)/2
        cylinder.translate(center)
        
        return cylinder
    
    def create_cone_at_point(self, start_point, end_point, radius, color):
        radius = radius*2
        cone_height = radius * 3  # Adjust the height of the cone relative to the radius of the cylinder
        cone = o3d.geometry.TriangleMesh.create_cone(radius=radius, height=cone_height)
        cone.paint_uniform_color(color)

        # Compute the direction vector and normalize it
        direction = (end_point - start_point) / np.linalg.norm(end_point - start_point)

        # Compute the rotation axis and angle
        z_axis = np.array([0, 0, 1])
        rotation_axis = np.cross(z_axis, direction)
        if np.linalg.norm(rotation_axis) > 0:
            rotation_axis /= np.linalg.norm(rotation_axis)
        rotation_angle = np.arccos(np.dot(z_axis, direction))

        # Apply the rotation
        cone.rotate(o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle), center=(0, 0, 0))

        # Translate to the end point
        cone.translate(end_point - direction * cone_height / 2)  # Adjust position to align cone tip to end_point

        return cone
    
    def key_callback(self, vis):
        #print(self.it)
        if self.seg_id > 14:
            #print('No more segments')
            return
        if self.forward:
            sampled_anchor_point = self.anchor_points[self.seg_id] + (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        else:
            sampled_anchor_point = self.anchor_points[self.seg_id] - (self.it*self.sample_dist)*self.intersection_lines[self.seg_id][0]/np.linalg.norm(self.intersection_lines[self.seg_id][0])
        [k, idx, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, self.radius)
        self.sampled_neighborhoods[self.it] = idx
        [k_color, idx_color, _] = self.pcd_tree.search_radius_vector_3d(sampled_anchor_point, 5)
        self.coloring_sampled_neighborhoods[self.it] = idx_color
        if k < 8:
            if self.forward:
                print('Point cloud boundary reached. Turning backwards')
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.it = 0
                self.forward = False
                self.eig_val_dict.clear()
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                return
            else:
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                self.forward = True
                print('Point cloud boundary reached. Moving on to next segment')
                self.seg_id += 1
                self.it = 0
                self.sampled_neighborhoods.clear()
                self.coloring_sampled_neighborhoods.clear()
                self.eig_val_dict.clear()
                return

        neighbor_coordinates = np.asarray(self.pcd.points)[idx[1:], :]

        mu = np.mean(neighbor_coordinates, axis=0)
        norm = neighbor_coordinates - mu
        cov = np.cov(norm.T)
        eig_val, eig_vec = np.linalg.eig(cov)
        sorted_idx = np.argsort(eig_val)[::-1]
        eig_val = eig_val[sorted_idx]
        eig_vec = eig_vec[:, sorted_idx]
        #print(eig_val)
        eig_val_norm = eig_val.copy()
        for z in range(len(eig_val)):
            eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))
        self.eig_val_dict[self.it] = eig_val_norm

        plane_dir = np.cross(eig_vec[:, 0], eig_vec[:, 1])
        #print(f'Plane dict: {self.plane_dir_dict}')
        if self.it == 0:
            self.plane_dir_dict[self.it] = plane_dir
        elif self.it > 0:
            previous_plane_dir = self.plane_dir_dict[self.it-1]
            if np.dot(plane_dir,previous_plane_dir)<0:
                plane_dir = -plane_dir
            self.plane_dir_dict[self.it] = plane_dir


        eigen_agg = 0
        angle_agg = 0
        if self.it > self.number_of_neighbors:
            for eigen_index in range(len(eig_val_norm)):
                for prev_iteration in range(self.number_of_neighbors):
                    #Calculate aggragate for the eigenvalues
                    eigen_diff = eig_val_norm[eigen_index] - self.eig_val_dict[self.it-(prev_iteration+1)][eigen_index]
                    eigen_agg += (eigen_diff/self.sample_dist)**2
                    
            for prev_iteration in range(self.number_of_neighbors):
                #Calculate aggragate for the planes of the ellipsoids
                dot_product = np.dot(plane_dir, self.plane_dir_dict[self.it-(prev_iteration+1)])
                norm_1 = np.linalg.norm(plane_dir)
                norm_2 = np.linalg.norm(self.plane_dir_dict[self.it-(prev_iteration+1)])
                plane_angle = np.arccos(np.clip(dot_product/(norm_1*norm_2), -1, 1))
                angle_agg += plane_angle
            angle_agg /= self.number_of_neighbors
            eigen_agg /= self.number_of_neighbors
            #print(angle_agg)
            #print(eigen_agg)
            if self.it == self.number_of_neighbors + 1:
                self.initial_eigen_agg = eigen_agg
                self.initial_angle_agg = angle_agg
                if forward:
                    edge_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f"Edge {self.seg_id}", depth=4).to_legacy()
                    edge_text_mesh.paint_uniform_color((1, 0, 0))
                    edge_text_location = self.anchor_points[self.seg_id]
                    edge_text_mesh.transform([[0.3, 0, 0, edge_text_location[0]+5], [0, 0.3, 0, edge_text_location[1]+5], [0, 0, 0.3, edge_text_location[2]+3], [0, 0, 0, 1]])
                    vis.add_geometry(edge_text_mesh)
            if eigen_agg > self.eigen_threshold or angle_agg > self.angle_threshold:
                if self.forward:
                    #print(f'broke edge moving forwards with initial angle_agg: {self.initial_angle_agg} and current angle_agg: {angle_agg}')
                    #print('')
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.it = 0
                    self.forward = False
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    return
                else:
                    #print(f'broke edge moving backwards with initial eigen_agg: {self.initial_eigen_agg} and current eigen_agg: {eigen_agg}')
                    #print('')
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                    self.it = 0
                    self.forward = True
                    self.eig_val_dict.clear()
                    self.plane_dir_dict.clear()
                    self.sampled_neighborhoods.clear()
                    self.coloring_sampled_neighborhoods.clear()
                    self.seg_id += 1
                    return
            else: 
                blue_color = plt.get_cmap("tab20")(0)
                if self.it > self.number_of_neighbors + 1:
                    np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it-1][1:], :] = self.pcd_color
                if self.ellipsoid_geometry:
                    self.toggle_ellipsoid(vis=vis)
                np.asarray(self.pcd.colors)[self.sampled_neighborhoods[self.it][1:], :] = (0.2,0.3,0.8)
                scale_factor = 4  # Adjust this factor to scale the length of the eigenvectors appropriately for visualization
                for line in self.line_geometry:
                    vis.remove_geometry(line, reset_bounding_box=False)
                self.line_geometry.clear()
                for i in range(3):  # Assuming 3D, so we have three principal components
                    start_point = mu
                    end_point = mu + np.sqrt(eig_val_norm[i]) * eig_vec[:, i] * scale_factor
                    color = [i == 0, i == 1, i == 2]  # Color the lines RGB based on principal component order
                    if np.linalg.norm(end_point - start_point) < 0.3:
                        end_point+=0.1* eig_vec[:, i]
                        radius = 0.04
                    else:
                        radius = 0.07  # Adjust the radius for thickness

                    # Create cylinder and add to visualizer
                    cylinder = self.create_cylinder_between_points(start_point, end_point, radius, color)
                    vis.add_geometry(cylinder)
                    self.line_geometry.append(cylinder)

                        # Create cone at the end point and add to visualizer
                    cone = self.create_cone_at_point(start_point, end_point, radius, color)
                    vis.add_geometry(cone)
                    self.line_geometry.append(cone)
                self.eigen_agg_text_mesh.clear()
                self.eigen_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Eigen agg = {eigen_agg:.2f}', depth=2).to_legacy()
                self.eigen_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.eigen_agg_text_location = sampled_anchor_point + [10,10,3]
                self.eigen_agg_text_mesh.transform([[0.2, 0, 0, self.eigen_agg_text_location[0]], [0, 0.2, 0, self.eigen_agg_text_location[1]], [0, 0, 0.2, self.eigen_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.eigen_agg_text_mesh)

                self.angle_agg_text_mesh.clear()
                self.angle_agg_text_mesh: TriangleMesh = o3d.t.geometry.TriangleMesh.create_text(f'Angle agg = {angle_agg:.2f}', depth=2).to_legacy()
                self.angle_agg_text_mesh.paint_uniform_color((1, 0, 0))
                self.angle_agg_text_location = sampled_anchor_point + [10,5,3]
                self.angle_agg_text_mesh.transform([[0.2, 0, 0, self.angle_agg_text_location[0]], [0, 0.2, 0, self.angle_agg_text_location[1]], [0, 0, 0.2, self.angle_agg_text_location[2]], [0, 0, 0, 1]])
                vis.add_geometry(self.angle_agg_text_mesh)

                view_ctl = vis.get_view_control()
                lookat = sampled_anchor_point
                zoom = 0.350000000000000002
                front = [-0.024106890455448116,-0.57254772319971181,0.81951690799604338]
                up =  [0.014828165865396817,0.81946017828866602,0.57294427451208185]
                view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
                view_ctl.set_up(up)      # Set the up direction of the camera
                view_ctl.set_front(front)  # Set the front direction of the camera
                view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera
                vis.update_geometry(self.pcd)
            
        self.it += 1

visualization = o3d.visualization.VisualizerWithKeyCallback()
state = CallbackState(anchor_points=anchor_points,pcd_tree=pcd_tree,eigen_threshold=eigen_threshold,number_of_neighbors=number_of_neighbors,sample_dist=sample_dist,
                      radius=radius,intersection_lines=intersection_lines, pcd=pcd, angle_threshold=angle_threshold)  
visualization.create_window()
visualization.get_render_option().background_color = np.asarray([0.95, 0.95, 0.95])
visualization.add_geometry(pcd)
#view_ctl = visualization.get_view_control()
#view_ctl.set_lookat(lookat)  # Set the point the camera is looking at
#view_ctl.set_up(up)      # Set the up direction of the camera
#view_ctl.set_front(front)  # Set the front direction of the camera
#view_ctl.set_zoom(zoom)          # Set the zoom factor of the camera

visualization.register_key_callback(263, state.toggle_ellipsoid)
visualization.register_key_callback(262, state.key_callback)
visualization.run()
visualization.destroy_window()